In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from Experiments.experiment_utils import load_experiment
import numpy as np
from scipy.optimize import minimize
import os

In [ ]:
import matplotlib.pylab as pylab
params = {'font.family':'serif', 
    'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
pylab.rcParams.update(params)

In [ ]:
def find_discrete_optimum(x, uncontrollables, objective_fun):
    xx, yy = np.meshgrid(np.arange(x.shape[0], dtype='int'), np.arange(uncontrollables.shape[0], dtype='int'))
    X = np.hstack((x[xx.flatten()], uncontrollables[yy.flatten()]))

    vals = objective_fun(X)
    vals_reshaped = vals.reshape(xx.shape)
    f_max = np.max(vals_reshaped, axis=0)
    return f_max

# Parameters for the synthetic experiments

In [ ]:
linestyles = ['-', '--', '-.', ':', '-', '--', '-.', '-', '-']
colors = ['tab:orange', 'tab:orange', 'tab:orange', 'tab:orange', 'firebrick', 'firebrick', 'firebrick', 'cornflowerblue']
alphas = [1., 0.8, 0.6, 0.4, 1., 0.7, 0.4, 1, 1]
flags = ['immediate', 'immediate', 'immediate', 'immediate', 'immediate', 'immediate', 'immediate', 'inference']
algorithms = ['MES', 'UCB', 'EI', 'KG', 'stableopt', 'stableopt', 'stableopt', 'RVES']
samples_vec = [1, 1, 1, 1, 1, 2, 4, 1]

# Hartmann 3D


In [ ]:
model, input_scaler, output_scaler, func_on_normalized, space, iterations, uncontrollables, objective_fun = load_experiment('hartmann_3d', 'RVES', 10);

# create a grid of the parameters
xx, yy = np.meshgrid(np.arange(space[0]['values'].shape[0], dtype='int'), np.arange(space[1]['values'].shape[0], dtype='int'))
X = np.hstack([space[0]['values'][xx.flatten()], space[1]['values'][yy.flatten()]])

vals = objective_fun(X)
vals_reshaped = vals.reshape(xx.shape)
f_opt = np.min(np.max(vals_reshaped, axis=0))
x_opt = X[(vals == f_opt).flatten(), :]
print('f_opt', f_opt)
print('x_opt', x_opt)

In [ ]:
experiment = 'hartmann_3d'

fig, axs = plt.subplots()

for idx, algorithm in enumerate(algorithms):        
    n_samples = samples_vec[idx]
    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm  + '/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm + '/'

    files = os.listdir(filepath)

    for ini in range(50):
        x_opt = x_opt
        fx_opt = f_opt

        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]
            
        df = pd.read_csv(filepath + filename[0])

        robust_immediate_regret = np.abs(find_discrete_optimum(df[['x_cand_0', 'x_cand_1']].to_numpy(), uncontrollables, objective_fun)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum(df[['x_opt_0', 'x_opt_1']].to_numpy(), uncontrollables, objective_fun)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_inference_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_immediate_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_ylabel('robust regret')
axs.set_xlabel('iteration')
axs.set_yscale('log')

fig.savefig('Plots/hartmann_3d.pdf', bbox_inches='tight')

# Sin plus linear

In [ ]:
model, input_scaler, output_scaler, func_on_normalized, space, iterations, uncontrollables, objective_fun = load_experiment('sin_plus_linear', 'RVES', 10);
# find the robust optimum

def optimization_objective(x, fun, uncontrollables):
    # combine the actual value of x with all uncontrollables
    x = np.atleast_2d(x)
    X = np.hstack((np.repeat(x, repeats=uncontrollables.shape[0], axis=0), uncontrollables))

    vals = fun(X)
    return np.max(vals, axis=0)

num_restarts = 50
restarts = np.random.rand(num_restarts, space[0]['dimensionality'])
bounds = [(0, 1) for _ in range(space[0]['dimensionality'])]

x_opts = np.zeros_like(restarts)
fx_opts = np.zeros(num_restarts)
for idx, restart in enumerate(restarts):
    res = minimize(optimization_objective, x0=restart, args=(func_on_normalized, space[1]['values']), bounds=bounds)
    x_opts[idx] = res.x
    fx_opts[idx] = res.fun

argmin_val = np.argmin(fx_opts)
x_opt = x_opts[argmin_val]
f_opt = fx_opts[argmin_val]

print(x_opt, f_opt)

In [ ]:
experiment = 'sin_plus_linear'

fig, axs = plt.subplots()

for idx, algorithm in enumerate(algorithms):        
    n_samples = samples_vec[idx]
    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm   +'/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm  + '/'

    files = os.listdir(filepath)

    for ini in range(50):
        x_opt = x_opt
        fx_opt = f_opt
        
        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]            
        df = pd.read_csv(filepath + filename[0])

        robust_immediate_regret = np.abs(find_discrete_optimum(df.x_cand_0.to_numpy().reshape(-1, 1), uncontrollables, objective_fun)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum(df.x_opt_0.to_numpy().reshape(-1, 1), uncontrollables, objective_fun)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_inference_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_immediate_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_ylabel('robust regret')
axs.set_xlabel('iteration')
axs.set_yscale('log')

fig.savefig('Plots/sin_plus_linear.pdf', bbox_inches='tight')

# Eggholder

In [ ]:
model, input_scaler, output_scaler, func_on_normalized, space, iterations, uncontrollables, objective_fun = load_experiment('eggholder', 'RVES', 10);
# find the robust optimum

def optimization_objective(x, fun, uncontrollables):
    # combine the actual value of x with all uncontrollables
    x = np.atleast_2d(x)
    X = np.hstack((np.repeat(x, repeats=uncontrollables.shape[0], axis=0), uncontrollables))

    vals = fun(X)
    return np.max(vals, axis=0)

num_restarts = 50
restarts = np.random.rand(num_restarts, space[0]['dimensionality'])
bounds = [(0, 1) for _ in range(space[0]['dimensionality'])]

x_opts = np.zeros_like(restarts)
fx_opts = np.zeros(num_restarts)
for idx, restart in enumerate(restarts):
    res = minimize(optimization_objective, x0=restart, args=(func_on_normalized, space[1]['values']), bounds=bounds)
    x_opts[idx] = res.x
    fx_opts[idx] = res.fun

argmin_val = np.argmin(fx_opts)
x_opt = x_opts[argmin_val]
f_opt = fx_opts[argmin_val].reshape(1, -1)

print(x_opt, f_opt)

In [ ]:
experiment = 'eggholder'

fig5, axs5 = plt.subplots()

for idx, algorithm in enumerate(algorithms):        
    n_samples = samples_vec[idx]
    inference_regrets = []
    immediate_regrets = []
    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm  + '/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm + '/'


    files = os.listdir(filepath)
    for ini in range(50):
        x_opt = x_opt
        fx_opt = f_opt
        
        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]
            
        df = pd.read_csv(filepath + filename[0])
        x_cand = input_scaler.transform(np.array([df.x_cand_0, df.x_cand_1]).T)
        x_opt = input_scaler.transform(np.array([df.x_opt_0, df.x_opt_1]).T)

        robust_immediate_regret = np.abs(find_discrete_optimum(x_cand[:, [0]], uncontrollables, objective_fun=func_on_normalized)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum(x_opt[:, [0]], uncontrollables, objective_fun=func_on_normalized)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_inference_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_immediate_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_xlabel('iteration')
axs.set_ylabel('robust regret')
axs.set_yscale('log')
fig5.savefig('Plots/eggholder.pdf', bbox_inches='tight')


# Branin

In [ ]:
model, input_scaler, output_scaler, func_on_normalized, space, iterations, uncontrollables, objective_fun = load_experiment('branin', 'RVES', 10);

def optimization_objective(x, fun, uncontrollables):
    # combine the actual value of x with all uncontrollables
    x = np.atleast_2d(x)
    X = np.hstack((np.repeat(x, repeats=uncontrollables.shape[0], axis=0), uncontrollables))

    vals = fun(X)
    return np.max(vals, axis=0)

num_restarts = 50
restarts = np.random.rand(num_restarts, space[0]['dimensionality'])
bounds = [(0, 1) for _ in range(space[0]['dimensionality'])]

x_opts = np.zeros_like(restarts)
fx_opts = np.zeros(num_restarts)
for idx, restart in enumerate(restarts):
    res = minimize(optimization_objective, x0=restart, args=(func_on_normalized, uncontrollables.reshape(-1, 1)), bounds=bounds)
    x_opts[idx] = res.x
    fx_opts[idx] = res.fun

argmin_val = np.argmin(fx_opts)
x_opt = x_opts[argmin_val]
f_opt = fx_opts[argmin_val].reshape(1, -1)

print(x_opt, f_opt)

In [ ]:
experiment = 'branin'

fig5, axs5 = plt.subplots()

for idx, algorithm in enumerate(algorithms):        
    n_samples = samples_vec[idx]
    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm  + '/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm + '/'

    files = os.listdir(filepath)

    for ini in range(50):
        x_opt = x_opt
        fx_opt = f_opt
        
        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]

        df = pd.read_csv(filepath + filename[0])

        x_cand = input_scaler.transform(np.array([df.x_cand_0, df.x_cand_1]).T)
        x_opt = input_scaler.transform(np.array([df.x_opt_0, df.x_opt_1]).T)

        robust_immediate_regret = np.abs(find_discrete_optimum(x_cand[:, [0]], uncontrollables, objective_fun=func_on_normalized)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum(x_opt[:, [0]], uncontrollables, objective_fun=func_on_normalized)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(1, df_robust_inference_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1])
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(1, df_robust_inference_regrets.shape[1], 5), df_robust_immediate_regrets.shape[1])
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_ylabel('robust regret')
axs.set_xlabel('iteration')
axs.set_xticks(np.arange(51, step=10));

axs.set_yscale('log')
fig5.savefig('Plots/branin.pdf',  bbox_inches='tight')

# Synthetic Polynomial

In [ ]:
model, input_scaler, output_scaler, func_on_normalized, space, iterations, uncontrollables, objective_fun = load_experiment('synthetic_polynomial', 'RVES', 10);
# find the robust optimum

def optimization_objective(x, fun, uncontrollables):
    # combine the actual value of x with all uncontrollables
    x = np.atleast_2d(x)
    X = np.hstack((np.repeat(x, repeats=uncontrollables.shape[0], axis=0), uncontrollables))

    vals = fun(X)
    return np.max(vals, axis=0)

num_restarts = 50
restarts = np.random.rand(num_restarts, space[0]['dimensionality'])
bounds = [(0, 1) for _ in range(space[0]['dimensionality'])]

x_opts = np.zeros_like(restarts)
fx_opts = np.zeros(num_restarts)
for idx, restart in enumerate(restarts):
    res = minimize(optimization_objective, x0=restart, args=(func_on_normalized, space[1]['values']), bounds=bounds, method='Nelder-Mead')
    x_opts[idx] = res.x
    fx_opts[idx] = res.fun

argmin_val = np.argmin(fx_opts)
x_opt = x_opts[argmin_val]
f_opt = fx_opts[argmin_val]

print(x_opt, f_opt)

In [ ]:
experiment = 'synthetic_polynomial'

fig5, axs5 = plt.subplots()

for idx, algorithm in enumerate(algorithms):        
    n_samples = samples_vec[idx]
    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm  + '/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm + '/'

    files = os.listdir(filepath)

    for ini in range(50):
        x_opt = x_opt
        fx_opt = f_opt
        
        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]

        df = pd.read_csv(filepath + filename[0])

        robust_immediate_regret = np.abs(find_discrete_optimum(df[['x_cand_0', 'x_cand_1']].to_numpy(), uncontrollables, func_on_normalized)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum(df[['x_opt_0', 'x_opt_1']].to_numpy(), uncontrollables, func_on_normalized)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_inference_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), label=algorithm + str(n_samples), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_immediate_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_ylabel('robust regret')
axs.set_xlabel('iteration')
axs.set_yscale('log')

fig.savefig('Plots/synthetic_poly.pdf',  bbox_inches='tight')


# Robot Pushing

In [ ]:
from Experiments.robot_pushing.robot_pushing_functions import robot_push_3d
from Experiments.robot_pushing.find_optima_robot_pushing import find_optima_robot_pushing

find_optima_robot_pushing()

results = np.load('optima_robot_pushing.npz')
x_opts = results['x_opts']
f_opts = results['f_opts']


In [ ]:
def find_discrete_optimum_robot(x, uncontrollables, objective):
    xx, yy = np.meshgrid(np.arange(x.shape[0], dtype='int'), np.arange(uncontrollables.shape[0], dtype='int'))
    X = np.hstack((x[xx.flatten()], uncontrollables[yy.flatten()]))

    vals = np.array([objective(xi[0], xi[1], xi[2], xi[3], xi[4]) for xi in X])
    vals_reshaped = vals.reshape(xx.shape)
    f_max = np.max(vals_reshaped, axis=0)
    return f_max

In [ ]:
experiment = 'robot_pushing'

fig, axs = plt.subplots()

for idx, algorithm in enumerate(algorithms):
    for ini in range(30):   
        x_opt = x_opts[ini]
        fx_opt = f_opts[ini]
    
        filename = [file for file in files if (('init_' + str(ini) + '_') in file)]

        df = pd.read_csv(filepath + filename[0])

        cand_location = np.array([df.x_cand_0, df.x_cand_1, df.x_cand_2, df.x_cand_3, df.x_cand_4]).T
        opt_location = np.array([df.x_opt_0, df.x_opt_1, df.x_opt_2, df.x_opt_3, df.x_opt_4]).T        

        robust_immediate_regret = np.abs(find_discrete_optimum_robot(cand_location, uncontrollables, robot_push_3d)-fx_opt.item())
        robust_inference_regret = np.abs(find_discrete_optimum_robot(opt_location, uncontrollables, robot_push_3d)-fx_opt.item())

        robust_immediate_regrets.append(robust_immediate_regret)
        robust_inference_regrets.append(robust_inference_regret)

    df_robust_inference_regrets = pd.DataFrame(robust_inference_regrets)
    df_robust_immediate_regrets = pd.DataFrame(robust_immediate_regrets)     
    n_samples = samples_vec[idx]

    robust_inference_regrets = []
    robust_immediate_regrets = []

    if algorithm == 'stableopt':
        filepath = 'Results/' + experiment +'/' + algorithm  + '/exploration_' + str(n_samples) + '/'
    else:
        filepath = 'Results/' + experiment +'/' + algorithm + '/'

    df_robust_inference_regrets = pd.read_csv('Results/robot_pushing/analysis/' +'robot_pushing_robust_inference_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)
    df_robust_immediate_regrets = pd.read_csv('Results/robot_pushing/analysis/' +'robot_pushing_robust_immediate_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)

    if flags[idx] == 'inference':
        axs.plot(df_robust_inference_regrets.median(), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_inference_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_inference_regrets.median()[indices]
        lowers = medians-df_robust_inference_regrets.quantile(0.25)[indices]
        uppers = df_robust_inference_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])
    else:
        axs.plot(df_robust_immediate_regrets.median(), color=colors[idx], linestyle=linestyles[idx], alpha=alphas[idx])          
        indices = np.append(np.arange(0, df_robust_immediate_regrets.shape[1], 10), df_robust_immediate_regrets.shape[1]-1)
        medians = df_robust_immediate_regrets.median()[indices]
        lowers = medians-df_robust_immediate_regrets.quantile(0.25)[indices]
        uppers = df_robust_immediate_regrets.quantile(0.75)[indices]-medians
        axs.errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors[idx], linestyle='', capsize=4, alpha=alphas[idx])

axs.set_xticks(np.arange(0, 101, 10))
axs.set_ylabel('robust regret')
axs.set_xlabel('iteration')
axs.legend(['MES', 'UCB', 'EI', 'KG', 'StableOpt1', 'StableOpt2', 'StableOpt4', 'RES'], bbox_to_anchor=(1.43, 1), loc='upper right'),

axs.set_yscale('log')
fig.savefig('Plots/robot_pushing.pdf', bbox_inches='tight')


# Within model comparison

In [ ]:
# find the optima of the functions used in the within model analysis
from Experiments.within_model.find_optima_within_model import find_optima_within_model

find_optima_within_model()

# analyses the results
from Experiments.within_model.within_model_analysis import analyse_within_model

analyse_within_model()

algorithms = ['MES', 'UCB', 'EI', 'KG', 'stableopt', 'stableopt', 'stableopt', 'RVES', 'RVES', 'RVES', 'RVES']
experiment = 'within_model'
samples_vec = [1, 1, 1, 1, 1, 2, 4, 1, 5, 10, 30] 
colors_within = ['tab:orange', 'tab:orange', 'tab:orange', 'tab:orange', 'firebrick', 'firebrick', 'firebrick', 'cornflowerblue', 'cornflowerblue', 'cornflowerblue', 'cornflowerblue']
alphas_within = [1., 0.8, 0.6, 0.4, 1., 0.8, 0.6, 1., 0.8, 0.6, 0.4]
linestyles_within = ['-', '--', '-.', ':', '-', '--', '-.', '-', '--', '-.', ':' ]
subplot_idx = [ 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1]
flags = ['immediate', 'immediate', 'inference', 'immediate', 'immediate', 'immediate', 'immediate', 'inference', 'inference', 'inference', 'inference']
labels = ['MES', 'UCB', 'EI', 'KG', 'StableOpt1', 'StableOpt2', 'StableOpt4', 'RES1', 'RES5', 'RES10', 'RES30']


fig, axs = plt.subplots(1, 3, sharey=True, figsize=(20, 5))

for idx, algorithm in enumerate(algorithms):
    n_samples = samples_vec[idx]

    df_inference_regrets = pd.read_csv('Results/within_model/analysis/within_model_inference_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)
    df_immediate_regrets = pd.read_csv('Results/within_model/analysis/within_model_immediate_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)
    df_robust_inference_regrets = pd.read_csv('Results/within_model/analysis/within_model_robust_inference_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)
    df_robust_immediate_regrets = pd.read_csv('Results/within_model/analysis/within_model_robust_immediate_regret_' + algorithm + str(n_samples) + '.csv', index_col=0)


    if flags[idx] == 'inference':
        axs[subplot_idx[idx]].plot(df_inference_regrets.median(), color=colors_within[idx], linestyle=linestyles_within[idx], alpha=alphas_within[idx])          
        indices = np.append(np.arange(0, df_inference_regrets.shape[1], 10), df_inference_regrets.shape[1]-1)
        medians = df_inference_regrets.median()[indices]
        lowers = medians-df_inference_regrets.quantile(0.25)[indices]
        uppers = df_inference_regrets.quantile(0.75)[indices]-medians
        axs[subplot_idx[idx]].errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors_within[idx], linestyle='', capsize=4, alpha=alphas_within[idx])
        axs[subplot_idx[idx]].set_ylabel('inference regret')
    else:
        axs[subplot_idx[idx]].plot(df_immediate_regrets.median(), color=colors_within[idx], linestyle=linestyles_within[idx], alpha=alphas_within[idx])   
        indices = np.append(np.arange(0, df_immediate_regrets.shape[1], 10), df_immediate_regrets.shape[1]-1)
        medians = df_immediate_regrets.median()[indices]
        lowers = medians-df_immediate_regrets.quantile(0.25)[indices]
        uppers = df_immediate_regrets.quantile(0.75)[indices]-medians
        axs[subplot_idx[idx]].errorbar(indices, medians, np.vstack((lowers, uppers)), color=colors_within[idx], linestyle='', capsize=4, alpha=alphas_within[idx])

        axs[subplot_idx[idx]].set_ylabel('immediate regret')

    axs[subplot_idx[idx]].set_xlabel('iteration')
    axs[subplot_idx[idx]].set_yscale('log')
    axs[subplot_idx[idx]].set_xticks(np.arange(76, step=10))

    axs[1].legend(labels[-4:])
    axs[0].legend(labels[:4])
    axs[2].legend(labels[4:7])

fig.savefig('Plots/within_model.pdf', bbox_inches='tight')
